We are going to clip some rasters into a big mosaic. 
- Make sure the import is multiple or simple
- if multiple set the folder where all the layers are
- do the clipping into a created folder of the same name

In [1]:
#Import the libraries
import os
import sys
from osgeo import gdal
import numpy as np
import glob


gdal.UseExceptions()

In [4]:
"""Set folder main location"""
path = r"Z:\demo-geo-stack\geoserver\data\rini\population"

In [6]:
"""Get a list of the files inside the folder"""
File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
for file in os.listdir(path):
    if file.endswith(".tif"):
        if file not in File_list:
            File_list.append(file)
    else:
        pass
print(File_list)

['usa_pop_tot_wgs84_comp.tif', 'usa_pop_tot_wgs84_comp_1.tif']


In [7]:
print(File_list[1:])

['usa_pop_tot_wgs84_comp_1.tif']


In [ ]:
"""Do the clipping of the file"""

"""We locate ourselves in our working directory"""
os.chdir(path)

"""We iterate over the files"""
file_snap_list = []
for file in File_list[1:]:
    datafile = gdal.Open(file)

    """We create the destination folder and set ourselves inside"""
    dataname = file.replace('.tif','')
    dir = os.path.join(path, dataname + "_mosaic")
    if not os.path.exists(dir):
        os.mkdir(dir)
    # mosaic_path = os.open(dataname + "_mosaic")
    os.chdir(dir)

    """Here we get the size of the file"""
    width = datafile.RasterXSize #columns
    height = datafile.RasterYSize #rows
    # define your tile size. It could be 256,512 or 1024. it depends on your scope
    tilesize = 10240
    """Here we start with the tiling"""
    for i in range(0, width, tilesize): #tilesize marks from where to where in width
        for j in range(0, height, tilesize): #tilesize in height
            #this is for the edge parts, so we don't get nodata from the borders
            w = min(i+tilesize, width) - i
            h = min(j+tilesize, height) - j
            #we preapre the sentence
            output_file = dataname + "_" + str(i) + "_" + str(j) + ".tif"
            file_snap_list.append(output_file)
            try:
                """Set the parameters of the clipping"""
                ds = gdal.Translate(output_file, datafile, format = 'GTiff', 
                # noData = -32768, 
                # outputType = gdal.GDT_Int16, 
                creationOptions = ['COMPRESS=DEFLATE', 'TILED=YES','COPY_SRC_OVERVIEWS=YES'], 
                srcWin = [str(i), str(j), str(w), str(h)])
                # print(ds)
            except RuntimeError:
                print("The script got an error")
                sys.exit(1)
    """We have to close the file in order to proceed for the next parts"""
    ds = None
    

In [ ]:
"""This is for testing, avoid it"""
dir = os.path.join(r"//akif.internal/public/z_resources/hydrosoil/hihydrosoil_ruben",dataname + "_mosaic").replace("\\","/")
current_path = os.getcwd()
print(current_path)
tiles=glob.glob(dir + '/*')
print(tiles)
for img in tiles:
    print(img)

In [ ]:
"""with this block, we delete tiles that include just nodata"""
# os.chdir(r"//akif.internal/public/z_resources/hydrosoil/hihydrosoil_ruben")
# dir = r"Z:\demo-geo-stack\geoserver\data\im-data-soil\isric-bdticm-m-250m-ll_mosaic_10240"

tiles=glob.glob(dir.replace("/","\\") + '/*')
for img in tiles:
    ds_filter = gdal.Open(img, gdal.GA_ReadOnly)
    ds_array=ds_filter.ReadAsArray()
    check=np.all(ds_array==0)
    if check:
        ds_filter = None #we have to close the file before deleting it
        # print(img)
        os.remove(img)
        continue

ds_array=None
np_where_img=None